<a href="https://colab.research.google.com/github/AndreMartins21/SISU-UFMG/blob/main/DFbruto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# - LENDO O PDF, MANIPULANDO AS STRINGS E CRIANDO O DATAFRAME

In [2]:
# Importando as Bibliotecas

!pip install PyPDF2

import PyPDF2 as PDF
import pandas as pd
import regex as re

In [72]:
# Dados

arquivo1 = '/content/drive/MyDrive/Colab Notebooks/DataSet Kaggle/CHAMADA_REGULAR.pdf'

arquivo2 = '/content/drive/MyDrive/Colab Notebooks/DataSet Kaggle/1°CHAMADA.pdf'

**<> Lendo o arquivo PDF, e salvando todas as páginas**

In [65]:
def ler_pdf(arquivo):
  pdf_lido = PDF.PdfFileReader(arquivo)

  num_page = pdf_lido.numPages

  paginas = []

  # Filtrando os dados dos arquivos, separando somente os cursos / turnos / modalidade / máximo / mínimo
  for cont in range(0, num_page):
    page = pdf_lido.getPage(cont)

    conteudo = page.extractText()

    # SEPARAR SOMENTE OS CURSOS E SUAS NOTAS
    inicio = conteudo.find("MínimoMáximo") + len("MínimoMáximo")

    # Se for a última página:
    if cont == num_page-1:
      fim = conteudo.find('Mínimogeral') - 6

    elif conteudo.find("21/04/2021") != -1: 
      fim = conteudo.find("21/04/2021")
    
    else:
      fim = conteudo.find("28/04/2021")

    paginas.append(conteudo[inicio:fim])
  return paginas

cham_regular = ler_pdf(arquivo1)

prim_chamada = ler_pdf(arquivo2)

In [69]:
prim_chamada[4][:300]

'CONTROLADORIA E FINANÇASMatutino3.1557,82557,82CONTROLADORIA E FINANÇASMatutino3.2639,32689,68CONTROLADORIA E FINANÇASMatutino4.2717,08761,82CONTROLADORIA E FINANÇASMatutinoAmplaconcorrência716,56781,84DESIGNNoturno1.1466,54578,92DESIGNNoturno1.2652,94686,90DESIGNNoturno2.2682,56743,24DESIGNNoturno3'

In [70]:
# SEPARANDO OS VALORES DE CADA COLUNA DO DATAFRAME

def sep_curso(string):
  
  # "6CIÊNCIAS CONTÁBEISNoturno"
  # Curso com maior nome tem 49 letras
  
  curso = r'[A-ZÍÃÓÇÔÊÉÁÂÚ\s()(Bacharelado)(Licenciatura)]{0,49}[VNMI]'
  find_curso = re.findall(curso, string)
  find_curso = list([x[:-1] for x in find_curso])
  return find_curso


def sep_turno(string):

  turno = r'Matutino|Noturno|Integral|Vespertino'

  find_turno = re.findall(turno, string)

  return find_turno

def sep_modalidade(string):
  #1.1 | Amplaconcorrência

  modalidade = r'\d{1}\.\d{1}|Amplaconcorrência'

  find_modalidade = re.findall(modalidade, string)

  return find_modalidade

def sep_minimo(string):
  #520,04

  minimo = r'\d{3},\d{3}'

  find_min = re.findall(minimo, string)
  find_min = list([x[:6] for x in find_min])

  return find_min

def sep_maximo(string):
  #04520

  maximo = r'\d{5},\d{2}'

  find_max = re.findall(maximo, string)
  find_max = list([x[2:] for x in find_max])

  return find_max

In [73]:
def data_frame(arq, line):
  '''
  Função que retornará cada página do pdf em formato de DataFrame
  '''
  
  line = arq[line]

  df_sisu = pd.DataFrame({
      "Curso": sep_curso(line),
      "Turno": sep_turno(line),
      "Modalidade": sep_modalidade(line),
      "Mínimo": sep_minimo(line),
      "Máximo": sep_maximo(line)
  })

  return df_sisu

data_frame(prim_chamada, 15).head()

,Curso,Turno,Modalidade,Mínimo,Máximo
0,TERAPIA OCUPACIONAL,Matutino,2.2,"666,92","715,84"
1,TERAPIA OCUPACIONAL,Matutino,3.1,"495,34","513,18"
2,TERAPIA OCUPACIONAL,Matutino,3.2,"622,32","659,74"
3,TERAPIA OCUPACIONAL,Matutino,4.1,"577,84","577,84"
4,TERAPIA OCUPACIONAL,Matutino,4.2,"670,74","705,00"


# PASSANDO O ARQUIVO PARA .XLSX

In [90]:
# Montando o dataframe completo

dfs = []
for arq in [cham_regular, prim_chamada]:
  
  df = data_frame(arq, 0)

  for cont in range(1, 16):  
    df = df.append(data_frame(arq, cont), ignore_index= True)

    # Transformando os valores de MÁXIMO e MÍNIMO em float
    df["Máximo"] = df["Máximo"].apply(lambda x: x.replace(",", "."))
    df["Mínimo"] = df["Mínimo"].apply(lambda x: x.replace(",", "."))

  dfs.append(df)

df_regular = dfs[0]

df_1chamada = dfs[1]

In [92]:
df_regular.head()

,Curso,Turno,Modalidade,Mínimo,Máximo
0,ADMINISTRAÇÃO,Matutino,1.1,520.04,520.04
1,ADMINISTRAÇÃO,Matutino,1.2,642.08,674.66
2,ADMINISTRAÇÃO,Matutino,2.1,649.12,649.12
3,ADMINISTRAÇÃO,Matutino,2.2,698.38,719.48
4,ADMINISTRAÇÃO,Matutino,3.1,559.88,559.88


In [93]:
df_1chamada.head()

,Curso,Turno,Modalidade,Mínimo,Máximo
0,ADMINISTRAÇÃO,Matutino,1.1,520.04,520.04
1,ADMINISTRAÇÃO,Matutino,1.2,638.12,674.66
2,ADMINISTRAÇÃO,Matutino,2.1,649.12,649.12
3,ADMINISTRAÇÃO,Matutino,2.2,667.98,719.48
4,ADMINISTRAÇÃO,Matutino,3.1,559.88,559.88


In [102]:
# Transformando em arquivo Excel:

excel = pd.ExcelWriter('Maximo_Minimo_UFMG.xlsx')

df_regular.to_excel(excel, sheet_name='Chamada Regular')
df_1chamada.to_excel(excel, sheet_name='1° Chamada')

excel.save()